In [217]:
import os

import numpy as np
import tensorflow as tf

In [218]:
os.chdir("/home/ec2-user/cs231n")
os.listdir()

['results', 'data', 'milestone']

# Load data

In [219]:
def load_data(dir: str) -> np.ndarray:
    orig_dir = os.getcwd()
    os.chdir(dir)
    files = os.listdir()
    arrays = []
    for in_file in files:
      arrays.append(np.load(in_file, allow_pickle=True))
    out = np.stack(arrays)
    out = np.transpose(out, (0, 2, 3, 1))
    out = tf.image.resize(out, [224, 224])
    os.chdir(orig_dir)
    return out.numpy()

In [220]:
pos = load_data("data/pos")
print(pos.shape)

(15, 224, 224, 3)


In [221]:
neg = load_data("data/neg")
print(neg.shape)

(33, 224, 224, 3)


## Training validation split


In [222]:
def gen_sampling_weights(y: np.ndarray) -> np.ndarray:
  """Inverse probability sampling weights."""
  n1 = sum(y == 1)
  n0 = sum(y == 0)
  p1 = n1 / (n1 + n0)
  p0 = 1 - p1
  w1 = 1. / p1
  w0 = 1. / p0
  out = np.ones_like(y)
  out[y == 1] = w1
  out[y == 0] = w0
  return out

In [223]:
train_prop = 0.6
n1 = pos.shape[0]
n0 = neg.shape[0]
cut1 = int(train_prop * n1)
cut0 = int(train_prop * n0)

In [224]:
pos_train = pos[:cut1, :]
pos_val = pos[cut1:, :]
neg_train = neg[:cut0, :]
neg_val = neg[cut0:, :]

In [225]:
x_train = np.concatenate((pos_train, neg_train), axis=0)
x_val = np.concatenate((pos_val, neg_val), axis=0)

In [226]:
y1_train = np.ones(pos_train.shape[0])
y0_train = np.zeros(neg_train.shape[0])
y_train = np.concatenate((y1_train, y0_train), axis=0)

y1_val = np.ones(pos_val.shape[0])
y0_val = np.zeros(neg_val.shape[0])
y_val = np.concatenate((y1_val, y0_val), axis=0)

In [227]:
weights_train = gen_sampling_weights(y_train)
weights_val = gen_sampling_weights(y_val)

# Generators

In [228]:
# Applies transformations to an input image.
transformer = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=360,
    horizontal_flip=True,
    vertical_flip=True,
)

In [229]:
# Generators.
train_generator = transformer.flow(
    x=x_train,
    y=y_train,
    batch_size=64,
    sample_weight=weights_train
)

val_generator = transformer.flow(
    x=x_val,
    y=y_val,
    batch_size=32,
    sample_weight=weights_val
)

## Model

In [230]:
# Resnet module.
resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=(224, 224, 3),
    pooling="avg"
)
resnet.trainable = False

In [231]:
# Model.
inputs = tf.keras.Input(shape=(224, 224, 3), name="input")
h = resnet(inputs, training=False)
outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(h)
model = tf.keras.Model(inputs, outputs)

In [232]:
# Compile.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc"),
        tf.keras.metrics.AUC(name="auc")
      ]
)

In [233]:
# Model summary.
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
output (Dense)               (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________


In [234]:
# Callbacks.
callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir="milestone"),
  tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
]

## Training

In [236]:
hist = model.fit(
    x=train_generator,
    validation_data=val_generator,
    validation_freq=10,
    epochs=200,
    callbacks=callbacks,
)

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 1.3855 - acc: 0.5714 - auc: 0.4737
Epoch 2/200
1/1 [==============================] - 2s 2s/step - loss: 1.3847 - acc: 0.6429 - auc: 0.5000
Epoch 3/200
1/1 [==============================] - 1s 1s/step - loss: 1.3846 - acc: 0.6786 - auc: 0.5000
Epoch 4/200
1/1 [==============================] - 1s 1s/step - loss: 1.3837 - acc: 0.7857 - auc: 0.5000
Epoch 5/200
1/1 [==============================] - 1s 1s/step - loss: 1.3851 - acc: 0.6429 - auc: 0.4737
Epoch 6/200
1/1 [==============================] - 1s 1s/step - loss: 1.3840 - acc: 0.8214 - auc: 0.5292
Epoch 7/200
1/1 [==============================] - 2s 2s/step - loss: 1.3857 - acc: 0.6429 - auc: 0.4211
Epoch 8/200
1/1 [==============================] - 1s 1s/step - loss: 1.3849 - acc: 0.5714 - auc: 0.5848
Epoch 9/200
1/1 [==============================] - 1s 1s/step - loss: 1.3845 - acc: 0.6786 - auc: 0.5848
Epoch 10/200
1/1 [==============================] - 3s 

In [238]:
model.evaluate(
    x=x_train,
    y=y_train
)

1/1 [==============================] - 2s 2s/step - loss: 0.6920 - acc: 0.7857 - auc: 0.6404


[0.6920239329338074, 0.7857142686843872, 0.6403509378433228]

In [239]:
model.evaluate(
    x=x_val,
    y=y_val
)

1/1 [==============================] - 1s 1s/step - loss: 0.6926 - acc: 0.6000 - auc: 0.6131


[0.6926261186599731, 0.6000000238418579, 0.613095223903656]

In [240]:
model.save("results/milestone_model")

INFO:tensorflow:Assets written to: results/milestone_model/assets


/home/ec2-user/miniconda3/envs/vxm/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
